### 초기창업 패키지

In [ ]:
import json
import re

from docx import Document
from docx.shared import Pt
from openai import OpenAI

from docx2pdf import convert

In [ ]:
client = OpenAI(api_key='YOUR_API_KEY')

In [ ]:
template = """
1. 항목 제목  
   1-1. 제목  
       1) 소제목  
          - 세부 항목 1  
          - 세부 항목 2  
          - 세부 항목 3  
       2) 소제목  
          - 세부 항목 1  
          - 세부 항목 2  
          - 세부 항목 3  

2. 항목 제목  
   2-1. 제목  
       1) 소제목  
          - 세부 항목 1  
          - 세부 항목 2  
          - 세부 항목 3  
       2) 소제목  
          - 세부 항목 1  
          - 세부 항목 2  
          - 세부 항목 3
          """

In [ ]:
Problem_template = """
1. 문제인식(Problem)

   1-1. 창업 아이템의 필요성
      - 현재 시장에서 해결되지 않은 주요 문제점과 해당 문제는 국내외에서 미치는 영향을 작성하세요.
      - 현재 시장에서 사용되는 기존 해결책(경쟁 제품/서비스)의 한계점과 문제들의 해결책을 작성하세요.
      - 소비자 혹은 타겟 고객의 불편함과 문제점을 구체적으로 작성하세요.
      - 기존 시장에서 유사한 해결책이 있었던 경우와 부족했던 부분을 작성하세요.
   예시) 1-2. 위에 내용에 관련된 내용들을 알맞게 작성하세요.
   """

Solution_template = """
2. 실현가능성(Solution) (2번 부터 작성.)

   2-1. 창업 아이템 현황 및 준비 정도
      - 제품·서비스의 필요성에 대한 문제를 인식하고, 해당 제품·서비스 개발·개선을 위해 본 사업에 신청하기 이전까지 기획, 추진한 이력 등에 대해 작성하세요.
      - 사업 신청 시점의 아이템 개발 단계별 현황, 목표한 시장 및 고객의 반응, 현재까지의 주요 사업화 성과 등 전반적인 현황을 작성하세요.
   2-2. 창업 아이템 실현 및 구체화 방안
      - 협약 기간 내 실현 및 구체화하고자 하는 개선 방안을 작성해주세요.
      - 경쟁사 제품·서비스와의 비교를 통해 파악된 문제점에 대해 자사의 보유역량을 기반으로 경쟁력, 차별성 등을 확보하기 위한 방안을 작성하세요.
      - 기타 개선·구체화하고자 하는 제품·서비스 관련 기술 보호 계획 등을 작성하세요.

   <사업추진 일정(협약기간 내)>
   예시) 표로 작성해주세요.
   |구분|추진 내용|추진 기간|세부 내용|

   <사업비 집행 계획>
   *"정부지원사업비는 총 사업비의 70%이하, 현금은 10%이상, 현물은 20% 이하로 작성하세요.(최대 1억원 한도 이내)"라고 적어주세요
   - 정부지원사업비가 총 사업비의 규정을 초과하지 않도록 주의하세요.
"""

ScaleUp_template = """
3. 성장전략(Scale-Up)

   3-1. 사업화 추진 전략
      - 유사 서비스 또는 제품을 제공하는 기업은 어떤 곳이며, 차별화되는 요소가 무엇인지 작성하세요.
      - 목표 시장과 해당 시장에서 진입 전략은 무엇인지 작성하세요.
      - 창업 아이템의 비즈니스 모델(수익 창출 방식)은 어떻게 구성되는지 작성하세요.
         예시) 구독 모델, 광고 모델, 데이터 판매, 폴랫폼 중개 등
      - 초기에 확보해야 할 주요 고객층은 누구이며, 어떻게 확보할 예정인지 작성하세요.
      - 투자 유치를 위한 전략(시드 펀딩, 투자 등), 제품·서비스 확장(글로벌 진출, B2B 협업 등)은 어떻게 계획을 하는지 작성하세요.
   3-2. ESG 경영 실천계획
      - 장기적으로 ESG(Environmental, Social, Governance) 가치를 어떻게 도입할 예정인지 작성하세요.
      - 환경 측면 : 친환경 원료 사용, 에너지 절감 계획 등 구체적으로 작성하세요.
      - 사회적 측면 : 지역 사회 기여, 사회 환원 활동 등 구체적으로 작성하세요.
      - 지배구조 측면 : 윤리적 경영 실천, 책임있는 투자 유치 등 구체적으로 작성하세요.

   <사업 전체 로드맵(일정)>
   예시) 표로 작성해주세요.
   |구분|추진 내용|추진 기간|세부 내용|
   |1|시제품 제작|2023.02 ~ 2023.04|시제품 설계|

   <사업추진 일정(전체 사업단계)>
   예시) 표로 작성해주세요.
   |구분|추진 내용|추진 기간|세부 내용|
   |1|시장 조사 및 초기 마케팅 활동|2023.04 ~ 2023.06|SNS 캠페인, 온라인 광고 진행|
"""

Team_template = """
4. 팀 구성(Team)
   
   4-1. 대표자 현황 및 역량
      - 대표자만 적어주세요.
      - 예시) 유사 경험, 지원사업 수행 이력, 관련 이수 현황, 관련 수상 실적 등을 작성하세요.
      - 사용자가 구체적으로 작성할 수 있도록 "예시"들을 작성해주세요.
   4-2. 팀원 구성 계획
      - 예시) 팀에서 보유 또는 보유할 예정인 장비·시설, 직원 역량(경력·학력, 기술력, 노하우 등) 작성하세요.
      - 예시) 협약기간 내 채용 예정인 인력에 대해서 작성하세요.
      - 예시) 팀원이 없을 경우 "없음" 이라고 작성하세요.
   
   <팀 구성 표>
   예시) 표로 작성해주세요.("예시"라는 문구를 보여주세요.)

   <협력 기관 현황 및 협업 방안>
   예시) 표로 작성해주세요.("예시"라는 문구를 보여주세요.)"""

## 1. Problem(문제인식)

In [ ]:
prompt1 = f"""
당신은 초기창업자를 위한 사업계획서 전문가입니다.
논리적이고 설득력 있는 사업계획서를 작성하세요.
문장은 간결하게, 핵심을 강조하며, 투자자 및 심사위원이 주목할 수 있도록 수치, 데이터, 시장 반응을 명확히 표현하세요.
이 작업은 창업자의 생계와 미래가 걸린 **절대적으로 중요한 프로젝트**입니다.
작성된 사업계획서의 완성도가 곧 **합격과 실패를 결정**하며, 이를 통해 창업자는 사업을 현실화하고 생존할 기회를 얻게 됩니다.
초안을 **작성할 때는 {loaded_data['문제인식']} 데이터를 기반으로만** 쓰며, **{Problem_template}**에 내용들을 순서에 맞게 구체적으로 작성해주세요.
양식은 아래 제공하는 제공하는 템플릿 양식을 참고하여 작성해주세요.

제공하는 템플릿 양식 : {template}

**반드시 지켜야 할 출력 규칙**
    1. 절대 #, * 기호 및 마크다운 스타일 절대 금지
    2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
    3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함. (ex. 출처: 통계청)
"""
completion1 = client.chat.completions.create(model='gpt-4o',
                                             messages=[{'role':'user', 'content':prompt1}],
                                             temperature=0.7,
                                             presence_penalty=0.8,
                                             frequency_penalty=0.5)
print(completion1.choices[0].message.content)

1. 문제인식

1-1. 창업 아이템의 필요성

1) 현재 시장 문제점
- 농산물 유통 시장은 구조적 비효율성으로 인해 생산자와 소비자 모두 가격 및 품질에서 손해를 보고 있습니다.
- 농산물 가격 변동은 예측이 어려워, 생산자들이 적절한 시기에 최적의 가격으로 판매하는데 어려움을 겪습니다. (출처: 한국농촌경제연구원)
- 기존 유통 채널들은 복잡하고 중간 마진이 커, 최종 소비자가격을 상승시키는 원인이 됩니다.

2) 기존 해결책의 한계
- 기존 온라인 플랫폼은 단순히 거래만 중개하며, 데이터 기반의 전략적 지원을 제공하지 않습니다.
- 일부 플랫폼에서 제공되는 가격 정보는 실시간 업데이트가 부족하여 신뢰도가 낮습니다.
- 이러한 서비스들은 개인화된 맞춤형 솔루션을 제공하지 않으며, 일반적인 정보만 제공합니다.

3) 창업 아이템의 해결 방안
- 데이터 기반 분석을 통해 효율적인 가격 책정과 판매 전략 수립을 지원합니다.
- 실시간 데이터 업데이트로 신속하고 정확한 시장 정보를 제공합니다.
- 맞춤형 전략 컨설팅 서비스를 통해 생산자의 시간과 비용 절감을 돕습니다.

4) 타겟 고객 불편 사항
- 농산물 온라인 판매자는 비효율적인 가격 책정 과정에서 많은 시간을 소모합니다.
- 판매전략 수립에 있어 명확한 기준이 없어 시행착오가 반복됩니다.
  
5) 유사 해결책과 그 부족함
- 일부 시스템은 단순 통계자료만 제공하며, 실행 가능한 전략을 제시하지 못합니다.
- 대다수의 서비스는 사용자 친화성이 떨어져 실제 활용도가 낮습니다.

1-2. 시장 영향 및 기회

1) 국내외 영향
- 농산물 유통 효율성 증대는 국내 시장 안정화뿐 아니라, 국제 무역 경쟁력 향상에도 기여할 수 있습니다. (출처: 농림축산식품부)
  
2) 성장 가능성
- 최근 5년간 온라인 농산물 거래량 증가 추세에 따라 관련 서비스 수요도 급증하고 있습니다. (출처: 통계청)

3) 경쟁 우위 요소 
- 본 서비스는 최신 AI 기술 적용으로 차별화된 데이터 분석 및 맞춤형 전략을 제시합니다.
 

## 2. Solution(실현가능성)

In [ ]:
prompt2 = f"""
당신은 초기창업자를 위한 사업계획서 전문가입니다.
논리적이고 설득력 있는 사업계획서를 작성하세요.
문장은 간결하게, 핵심을 강조하며, 투자자 및 심사위원이 주목할 수 있도록 수치, 데이터, 시장 반응을 명확히 표현하세요.
이 작업은 창업자의 생계와 미래가 걸린 **절대적으로 중요한 프로젝트**입니다.
작성된 사업계획서의 완성도가 곧 **합격과 실패를 결정**하며, 이를 통해 창업자는 사업을 현실화하고 생존할 기회를 얻게 됩니다.
초안을 **작성할 때는 {loaded_data['실현가능성']} 데이터를 기반으로만** 쓰며, **{Solution_template}**에 내용들을 순서에 맞게 참고해주세요.
양식은 아래 제공하는 제공하는 템플릿 양식을 참고하여 작성해주세요.

제공하는 템플릿 양식 : {template}

**반드시 지켜야 할 출력 규칙**
    1. 절대 #, * 기호 및 마크다운 스타일 절대 금지
    2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
    3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함. (ex. 출처: 통계청)
"""
completion2 = client.chat.completions.create(model='gpt-4o',
                                             messages=[{'role':'user', 'content':prompt2}],
                                             temperature=0.7,
                                             presence_penalty=0.8,
                                             frequency_penalty=0.5)
print(completion2.choices[0].message.content)

2. 실현가능성(solution)

2-1. 창업 아이템의 개발 계획
1) 제품/서비스 개요
- 본 서비스는 농산물 온라인 판매자에게 효율적인 가격 책정과 시간 단축을 지원하는 플랫폼입니다.
- 핵심 기능으로는 품목별, 판매 단위별 평균가 추이 확인 및 빠른 가격 설정 도구를 제공합니다.
- 기술적 접근 방식으로는 데이터 분석 알고리즘과 사용자 중심의 UI/UX 디자인을 적용합니다.

2) 개발 일정 및 목표
- 2023년 12월부터 문제 정의 및 가설 검증을 위한 초기 조사 시작.
- 2024년 1분기까지 와이어프레임 및 프로토타입 완료 예정.
- 2024년 중반까지 사용성 테스트를 통해 UI/UX 개선 후 최종 서비스 출시.

3) 차별화 포인트 및 경쟁력 확보 전략
- 차별화된 데이터 분석 기반으로 유의미한 정보 제공.
- 고객 맞춤형 데이터 가공 서비스를 통한 부가가치 창출로 경쟁력 강화.

<사업추진 일정(협약기간 내)>

| 구분 | 추진 내용 | 추진 기간 | 세부 내용 |
|------|-----------|------------|-------------|
| 문제 정의 및 조사 | 설문조사, 심층 인터뷰 진행 | 2023년 12월 - 2024년 1월 | 농산물 온라인 판매자 대상 |
| 기획 완료 | 유저 플로우, IA 구축 | 2024년 2월 - 3월 | 시장 조사 기반 |
| 개발 단계 | 와이어프레임, 프로토타입 제작 | 2024년 3월 - 5월 | 사용성 테스트 포함 |
| UI/UX 개선 | 최종 서비스 출시 준비 | 2024년 6월 - 하반기 | 피드백 반영 |

<사업비 집행 계획>

"정부지원사업비는 총 사업비의 70% 이하, 현금은 10% 이상, 현물은 20% 이하로 작성하세요.(최대 1억원 이내)"

구분            항목       금액(천 원)      세부 내역  
정부지원사업비 인건비    OOO           프로젝트 팀 구성  
                 개발비   OOO          소프트웨어 라

## 3. Scale-up(성장전략)

In [ ]:
prompt3 = f"""
당신은 초기창업자를 위한 사업계획서 전문가입니다.
논리적이고 설득력 있는 사업계획서를 작성하세요.
문장은 간결하게, 핵심을 강조하며, 투자자 및 심사위원이 주목할 수 있도록 수치, 데이터, 시장 반응을 명확히 표현하세요.
이 작업은 창업자의 생계와 미래가 걸린 **절대적으로 중요한 프로젝트**입니다.
작성된 사업계획서의 완성도가 곧 **합격과 실패를 결정**하며, 이를 통해 창업자는 사업을 현실화하고 생존할 기회를 얻게 됩니다.
초안을 **작성할 때는 {loaded_data['성장전략']} 데이터를 기반으로만** 쓰며, **{ScaleUp_template}**에 내용들을 순서에 맞게 참고해주세요.
양식은 아래 제공하는 제공하는 템플릿 양식을 참고하여 작성해주세요.

제공하는 템플릿 양식 : {template}

**반드시 지켜야 할 출력 규칙**
    1. 절대 #, * 기호 및 마크다운 스타일 절대 금지
    2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
    3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함. (ex. 출처: 통계청)
"""

completion3 = client.chat.completions.create(model='gpt-4o',
                                             messages=[{'role':'user', 'content':prompt3}],
                                             temperature=0.7,
                                             presence_penalty=0.8,
                                             frequency_penalty=0.5)
print(completion3.choices[0].message.content)

3. 성장전략(Scale-Up)

3-1. 사업화 추진 전략  
    1) 경쟁사 분석 및 차별화 요소
        - 유사 서비스 제공 기업: 농산물 위탁 판매 플랫폼 A, B
        - 차별화 요소: 셀팜은 빅데이터 기반의 전략 추천 기능을 통해 실시간 시장 분석과 즉각적인 실행 가능 액션 제공
        - 추가 차별화: 사용자 친화적 인터페이스와 맞춤형 데이터 인사이트 제공

    2) 시장 진입 전략 및 비즈니스 모델
        - 목표 시장: 20~30대 국내 농산물 위탁판매자 (출처: 통계청)
        - 진입 전략: 초기 유료 서비스로 부분서비스 유료화, 다양한 이커머스 플랫폼 데이터 분석으로 서비스 고도화 
        - 비즈니스 모델: 구독 모델 및 광고 수익 모델을 채택하여 지속 가능한 수익 창출 계획

    3) 고객층 확보 및 투자 유치 계획
        - 주요 고객층: 초기 단계에서는 소규모 농산물 판매자 집중 공략
        - 확보 방법: SNS 캠페인과 온라인 광고 활용으로 브랜드 인지도 확산 
        - 투자 유치 전략: 시드 펀딩을 통한 초기 자본 조달 후 B2B 협업을 통한 글로벌 시장 확장 계획

3-2. ESG 경영 실천계획  
    1) 환경 측면
        - 친환경 원료 사용 확대 및 에너지 절감 기술 도입 추진 
        - 탄소 배출 최소화를 위한 내부 프로세스 최적화
    
    2) 사회적 측면
        - 지역 농업 커뮤니티와의 파트너십 구축으로 사회 기여 강화
        - 정기적인 사회 환원 활동 실시로 기업 책임 경영 실현
    
    3) 지배구조 측면
        - 윤리적 경영 방침 수립 및 수행, 책임 있는 투자자 선정 기준 마련 
        - 투명한 의사 결정 구조 확립으로 신뢰성 강화

<사업 전체 로드맵(일정)> (예시라는 문구를 보여주세요.)

|구분|추진 내용|추진 기간|세부 내용|
|---|---|---|-

## 4. Team(팀 구성)

In [ ]:
prompt4 = f"""
당신은 초기창업자를 위한 사업계획서 전문가입니다.
논리적이고 설득력 있는 사업계획서를 작성하세요.
문장은 간결하게, 핵심을 강조하며, 투자자 및 심사위원이 주목할 수 있도록 수치, 데이터, 시장 반응을 명확히 표현하세요.
이 작업은 창업자의 생계와 미래가 걸린 **절대적으로 중요한 프로젝트**입니다.
작성된 사업계획서의 완성도가 곧 **합격과 실패를 결정**하며, 이를 통해 창업자는 사업을 현실화하고 생존할 기회를 얻게 됩니다.
초안을 **작성할 때는 {loaded_data['팀 구성']} 데이터를 기반으로만** 쓰며, **{Team_template}**에 내용들을 순서에 맞게 참고해주세요.
양식은 아래 제공하는 제공하는 템플릿 양식을 참고하여 작성해주세요.

제공하는 템플릿 양식 : {template}

**반드시 지켜야 할 출력 규칙**
    1. 절대 #, * 기호 및 마크다운 스타일 절대 금지
    2. 항목 표시 시 "1)", "2)" 형식만 사용 (숫자+닫는 괄호)
    3. 데이터 출처 명시 : 모든 통계 수치에 출처를 포함. (ex. 출처: 통계청)
"""

completion4 = client.chat.completions.create(model='gpt-4o',
                                             messages=[{'role':'user', 'content':prompt4}],
                                             temperature=0.7,
                                             presence_penalty=0.8,
                                             frequency_penalty=0.5)
print(completion4.choices[0].message.content)

4. 팀 구성  
4-1. 대표자 현황 및 역량  
1) 유사 경험  
- 이재호 대표는 5년간 IT 스타트업에서 프로젝트 매니저로 근무하며 다수의 성공적인 서비스 런칭을 주도하였습니다.

2) 지원사업 수행 이력  
- 정부 주관 창업 지원 프로그램에 선정되어 1억원의 자금을 유치한 경험이 있습니다.

3) 관련 이수 현황  
- 서울대학교 경영학과 졸업, 카이스트 기술경영 석사 과정 수료 중입니다.

4) 관련 수상 실적  
- 2021년 '대한민국 창업대전'에서 우수상을 수상했습니다.

4-2. 팀원 구성 계획   
1) 보유 또는 보유할 예정인 장비·시설 
- 현재 클라우드 서버 및 데이터 분석 소프트웨어를 활용 중이며, 추가로 최신 AI 도구를 제공받을 예정입니다.

2) 직원 역량(경력·학력, 기술력, 노하우 등)
- 핵심 개발자는 컴퓨터 공학 박사 출신으로 AI 분야에서 10년 이상의 경력을 보유하고 있습니다.
- 마케팅 담당자는 글로벌 브랜드 관리 경험이 있는 MBA 출신입니다.

3) 협약기간 내 채용 예정인 인력
- 향후 UX/UI 디자이너와 데이터 사이언티스트를 각 1명씩 채용할 계획입니다.
      
<팀 구성 표>  
예시:  
| 이름   | 역할         | 경력            | 학력           |
|--------|--------------|-----------------|----------------|
| 이재호 | 대표         | IT 스타트업 PM 5년 | 서울대 경영학과 졸업 |
| 없음   |              |                 |                |

<협력 기관 현황 및 협업 방안>   
예시:   
| 기관명       | 협업 내용                      | 기대 효과                     |
|--------------|----------------------------------|--------------------------

In [23]:
# 생성된 사업계획서 초안 출력
사업계획서_초안1 = completion1.choices[0].message.content
사업계획서_초안1 = 사업계획서_초안1.replace("### ", "")  # 자동으로 붙은 '### '을 제거

사업계획서_초안2 = completion2.choices[0].message.content
사업계획서_초안2 = 사업계획서_초안2.replace("### ", "")

사업계획서_초안3 = completion3.choices[0].message.content
사업계획서_초안3 = 사업계획서_초안3.replace("### ", "")

사업계획서_초안4 = completion4.choices[0].message.content
사업계획서_초안4 = 사업계획서_초안4.replace("### ", "")

In [24]:
ccp_dict = {'초안1': 사업계획서_초안1,
            '초안2': 사업계획서_초안2,
            '초안3': 사업계획서_초안3,
            '초안4': 사업계획서_초안4}

In [ ]:
# 딕셔너리 데이터를 JSON 파일로 저장
with open('business_plan/초기창업패키지_사업계획서.json', 'w', encoding='utf-8') as file:
    json.dump(ccp_dict, file, ensure_ascii=False, indent=4)

In [ ]:
# 예비창업패키지
# 초기창업패키지

# 문서 생성
doc = Document()

# 정규식을 사용하여 각 초안을 띄어쓰기 기준으로 나눔
for _, value in ccp_dict.items():
    re_text = re.split(r'\n\n', value)

    for idx in re_text:

        if '|' in idx:
            text = idx.split('|')
            doc.add_paragraph(text[0])

            data = text[1:-1]

            # '---'가 포함된 항목 제거 및 공백 정리
            filtered_data = [item.strip() for item in data if '-' not in item and item]

            # 데이터를 표 형식(2D 리스트)으로 변환
            table_data = []
            row = []

            for item in filtered_data:
                if item == "":  # 빈 문자열(\n 변환)일 경우 새로운 행 시작
                    if row:  # 현재까지 저장된 행이 있다면 추가
                        table_data.append(row)
                        row = []
                else:
                    row.append(item)

            # 마지막 행 추가 (루프 종료 후 남은 데이터 처리)
            if row:
                table_data.append(row)

            # 첫 번째 행을 자동으로 컬럼명으로 설정
            if table_data:
                headers = table_data.pop(0)  # 첫 번째 행을 컬럼명으로 설정
                table_data.insert(0, headers)  # 다시 삽입 (명확하게 구분하기 위함)


            # 표 생성 (행 개수는 데이터 행 개수, 열 개수는 첫 번째 행의 열 개수 기준)
            table = doc.add_table(rows=len(table_data), cols=len(table_data[0]))

            # 표 스타일 적용 (선택 사항: 표 테두리 추가)
            table.style = "Table Grid"

            # 표 데이터 삽입
            for row_idx, row_data in enumerate(table_data):
                for col_idx, cell_data in enumerate(row_data):
                    cell = table.cell(row_idx, col_idx)
                    cell.text = str(cell_data)  # 데이터 삽입
                    
                    # 첫 번째 행(컬럼명)은 굵게 표시
                    if row_idx == 0:
                        for paragraph in cell.paragraphs:
                            for run in paragraph.runs:
                                run.bold = True  # 글씨 굵게
                                run.font.size = Pt(12)  # 글씨 크기 조정 (선택 사항)

            doc.add_paragraph(text[-1])

        else:
            doc.add_paragraph(idx)

    doc.add_paragraph('\n\n') # 나눴던 구분자를 다시 집어넣음
    
# 파일 저장
doc.save("business_plan/초기창업패키지_초안.docx")

print("Word 파일이 성공적으로 생성되었습니다!")

In [ ]:
convert("business_plan/초기창업패키지_초안.docx", "business_plan/초기창업패키지_초안.pdf") # 워드 파일을 pdf로 변환